In [ ]:
# Development tool for parsing 9th column in GTF file.



In [3]:
import pandas as pd
import numpy as np

In [14]:
# Load file into DataFrame
gtffile = '/Users/jas971/PycharmProjects/pythonProject/ubkg-etl/generation_framework/owlnets_output/GENCODE/gencode.v41.annotation.gtf'

dfAnnotation = pd.read_csv(gtffile,comment='#',sep='\t',nrows=10)
# Set columns to match the GTF specification.
dfAnnotation.columns=['chromosome_name','annotation_source','feature_type','genomic_start_location','genomic_end_location','score','genomic_strand','genomic_phase','column_9']


In [15]:
# Split the key/value pairs in column 9 into columns, using the colon delimiter.
dfAnnotation['column_9']=dfAnnotation['column_9'].str.replace('"', '')
dfSplit_level_1 = dfAnnotation['column_9'].str.split(';',expand=True)
# Normalize empty column empty values to NaN.
dfSplit_level_1 = dfSplit_level_1.replace({'None': np.nan}).replace({None:np.nan}).replace({'':np.nan})
# Remove completely empty columns.
dfSplit_level_1 = dfSplit_level_1.dropna(axis=1,how='all')


In [16]:
dfSplit_level_1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,gene_id ENSG00000223972.5,transcript_id ENST00000456328.2,gene_type transcribed_unprocessed_pseudogene,gene_name DDX11L1,transcript_type processed_transcript,transcript_name DDX11L1-202,level 2,transcript_support_level 1,hgnc_id HGNC:37102,tag basic,havana_gene OTTHUMG00000000961.2,havana_transcript OTTHUMT00000362751.1,NaN,NaN,NaN,NaN,NaN
1,gene_id ENSG00000223972.5,transcript_id ENST00000456328.2,gene_type transcribed_unprocessed_pseudogene,gene_name DDX11L1,transcript_type processed_transcript,transcript_name DDX11L1-202,exon_number 1,exon_id ENSE00002234944.1,level 2,transcript_support_level 1,hgnc_id HGNC:37102,tag basic,havana_gene OTTHUMG00000000961.2,havana_transcript OTTHUMT00000362751.1,NaN,NaN,NaN
2,gene_id ENSG00000223972.5,transcript_id ENST00000456328.2,gene_type transcribed_unprocessed_pseudogene,gene_name DDX11L1,transcript_type processed_transcript,transcript_name DDX11L1-202,exon_number 2,exon_id ENSE00003582793.1,level 2,transcript_support_level 1,hgnc_id HGNC:37102,tag basic,havana_gene OTTHUMG00000000961.2,havana_transcript OTTHUMT00000362751.1,NaN,NaN,NaN
3,gene_id ENSG00000223972.5,transcript_id ENST00000456328.2,gene_type transcribed_unprocessed_pseudogene,gene_name DDX11L1,transcript_type processed_transcript,transcript_name DDX11L1-202,exon_number 3,exon_id ENSE00002312635.1,level 2,transcript_support_level 1,hgnc_id HGNC:37102,tag basic,havana_gene OTTHUMG00000000961.2,havana_transcript OTTHUMT00000362751.1,NaN,NaN,NaN
4,gene_id ENSG00000223972.5,transcript_id ENST00000450305.2,gene_type transcribed_unprocessed_pseudogene,gene_name DDX11L1,transcript_type transcribed_unprocessed_pseud...,transcript_name DDX11L1-201,level 2,transcript_support_level NA,hgnc_id HGNC:37102,ont PGO:0000005,ont PGO:0000019,tag basic,tag Ensembl_canonical,havana_gene OTTHUMG00000000961.2,havana_transcript OTTHUMT00000002844.2,NaN,NaN
5,gene_id ENSG00000223972.5,transcript_id ENST00000450305.2,gene_type transcribed_unprocessed_pseudogene,gene_name DDX11L1,transcript_type transcribed_unprocessed_pseud...,transcript_name DDX11L1-201,exon_number 1,exon_id ENSE00001948541.1,level 2,transcript_support_level NA,hgnc_id HGNC:37102,ont PGO:0000005,ont PGO:0000019,tag basic,tag Ensembl_canonical,havana_gene OTTHUMG00000000961.2,havana_transcript OTTHUMT00000002844.2
6,gene_id ENSG00000223972.5,transcript_id ENST00000450305.2,gene_type transcribed_unprocessed_pseudogene,gene_name DDX11L1,transcript_type transcribed_unprocessed_pseud...,transcript_name DDX11L1-201,exon_number 2,exon_id ENSE00001671638.2,level 2,transcript_support_level NA,hgnc_id HGNC:37102,ont PGO:0000005,ont PGO:0000019,tag basic,tag Ensembl_canonical,havana_gene OTTHUMG00000000961.2,havana_transcript OTTHUMT00000002844.2
7,gene_id ENSG00000223972.5,transcript_id ENST00000450305.2,gene_type transcribed_unprocessed_pseudogene,gene_name DDX11L1,transcript_type transcribed_unprocessed_pseud...,transcript_name DDX11L1-201,exon_number 3,exon_id ENSE00001758273.2,level 2,transcript_support_level NA,hgnc_id HGNC:37102,ont PGO:0000005,ont PGO:0000019,tag basic,tag Ensembl_canonical,havana_gene OTTHUMG00000000961.2,havana_transcript OTTHUMT00000002844.2
8,gene_id ENSG00000223972.5,transcript_id ENST00000450305.2,gene_type transcribed_unprocessed_pseudogene,gene_name DDX11L1,transcript_type transcribed_unprocessed_pseud...,transcript_name DDX11L1-201,exon_number 4,exon_id ENSE00001799933.2,level 2,transcript_support_level NA,hgnc_id HGNC:37102,ont PGO:0000005,ont PGO:0000019,tag basic,tag Ensembl_canonical,havana_gene OTTHUMG00000000961.2,havana_transcript OTTHUMT00000002844.2
9,gene_id ENSG00000223972.5,transcript_id ENST00000450305.2,gene_type transcribed_unprocessed_pseudogene,gene_name DDX11L1,transcript_type transcribed_unprocessed_pseud...,transcript_name DDX11L1-201,exon_number 5,exon_id ENSE00001746346.2,level 2,transcript_support_level NA,hgnc_id HGNC:37102,ont PGO:0000005,ont PGO:0000019,tag basic,tag Ensembl_canonical,hava

In [18]:
search_key = 'tag'
listval = []

# Key/value pairs do not have static locations--i.e., a key/value pair may be in column X in one row and column Y
# in another. 
# Furthermore, some keys repeat in the same row--e.g., tag, ont.


# Build a series of values for a particular key by looking through all columns.

# Example:
#     columns
# row       1         2         3       4
# 0         x 20
# 1         x 30
# 2                    x 40
# 3         a 99
# 4         y 25       z 30     x 50    x 60

# result for x
# 0 20
# 1 30
# 2 40
# 3 
# 4 50,60

for col in dfSplit_level_1:
    
    # Split each key/value pair column into separate key and value columns, using the space delimiter.
    # The strip function removes leading spaces that can be mistaken for delimiters.
    dfSplit_level_2 = dfSplit_level_1[col].str.strip()
    dfSplit_level_2 = dfSplit_level_2.str.split(' ',expand=True)
   
    # Obtain values that correspond to the search key.
    # In general, there are multiple values for a key on a row.
    dfSplit_level_2 = dfSplit_level_2[dfSplit_level_2[0]==search_key]
        
    if dfSplit_level_2.shape[0] > 0:
        listval.append(dfSplit_level_2)
        
if len(listval)>0:
    # Build the entire list of values for the key. 
    dfReturn = pd.concat(listval)
    # Concatenate multiple values that appear for the key in a row.
    # The column name for the value is 1, based on the earlier split.
    dfReturn = dfReturn.reset_index(names='rows')
    dfReturn = dfReturn.groupby('rows').agg({1:lambda x: ','.join(x)})
    sReturn = dfReturn[1]
else:
    sReturn = pd.Series(index=dfAnnotation.index.copy(),dtype='str')
    
print (sReturn)



rows
0                      basic
1                      basic
2                      basic
3                      basic
4    basic,Ensembl_canonical
5    basic,Ensembl_canonical
6    basic,Ensembl_canonical
7    basic,Ensembl_canonical
8    basic,Ensembl_canonical
9    basic,Ensembl_canonical
Name: 1, dtype: object
